In [ ]:
"""
SCRIPT COMPLET AUTONOME - AGENT DE TRADING AVANCÉ
Tout-en-un : Prédiction, Analyse, Décision d'Investissement
"""
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import time
import warnings
warnings.filterwarnings('ignore')

class AdvancedTradingAgent:
    def __init__(self, symbol='AAPL'):
        """
        Agent de trading avancé avec haute corrélation
        
        Args:
            symbol: Symbole boursier (ex: AAPL, MSFT, GOOGL)
        """
        self.symbol = symbol
        self.current_position = 0  # 0: neutre, 1: long, -1: short
        self.entry_price = 0
        self.decision_history = []
        self.performance_history = []
        
        # Paramètres avancés
        self.lookback_period = 60  # Période de lookback pour l'analyse
        self.confidence_threshold = 0.75  # Seuil de confiance minimum
        
    def get_historical_data(self, period='1y', interval='1d'):
        """Récupère les données historiques"""
        try:
            ticker = yf.Ticker(self.symbol)
            data = ticker.history(period=period, interval=interval)
            return data
        except:
            return None
    
    def get_current_price(self):
        """Récupère le prix actuel"""
        try:
            ticker = yf.Ticker(self.symbol)
            data = ticker.history(period='1d', interval='1m')
            if not data.empty:
                return data['Close'].iloc[-1]
            return None
        except:
            return None
    
    def calculate_advanced_indicators(self, data):
        """Calcule des indicateurs techniques avancés"""
        if len(data) < 50:
            return None
        
        # Moyennes mobiles multiples
        data['sma_5'] = data['Close'].rolling(5).mean()
        data['sma_10'] = data['Close'].rolling(10).mean()
        data['sma_20'] = data['Close'].rolling(20).mean()
        data['sma_50'] = data['Close'].rolling(50).mean()
        data['ema_12'] = data['Close'].ewm(span=12).mean()
        data['ema_26'] = data['Close'].ewm(span=26).mean()
        
        # RSI avec périodes multiples
        for period in [14, 21, 30]:
            delta = data['Close'].diff()
            gain = (delta.where(delta > 0, 0)).rolling(period).mean()
            loss = (-delta.where(delta < 0, 0)).rolling(period).mean()
            rs = gain / loss
            data[f'rsi_{period}'] = 100 - (100 / (1 + rs))
        
        # MACD
        data['macd'] = data['ema_12'] - data['ema_26']
        data['macd_signal'] = data['macd'].ewm(span=9).mean()
        data['macd_histogram'] = data['macd'] - data['macd_signal']
        
        # Bollinger Bands
        data['bb_middle'] = data['Close'].rolling(20).mean()
        bb_std = data['Close'].rolling(20).std()
        data['bb_upper'] = data['bb_middle'] + (bb_std * 2)
        data['bb_lower'] = data['bb_middle'] - (bb_std * 2)
        data['bb_width'] = (data['bb_upper'] - data['bb_lower']) / data['bb_middle']
        data['bb_position'] = (data['Close'] - data['bb_lower']) / (data['bb_upper'] - data['bb_lower'])
        
        # Stochastic Oscillator
        low_14 = data['Low'].rolling(14).min()
        high_14 = data['High'].rolling(14).max()
        data['stoch_k'] = 100 * ((data['Close'] - low_14) / (high_14 - low_14))
        data['stoch_d'] = data['stoch_k'].rolling(3).mean()
        
        # Volume indicators
        data['volume_sma'] = data['Volume'].rolling(20).mean()
        data['volume_ratio'] = data['Volume'] / data['volume_sma']
        data['volume_price_trend'] = data['Volume'] * data['Close'].pct_change()
        
        # Price momentum
        data['momentum_5'] = data['Close'].pct_change(5)
        data['momentum_10'] = data['Close'].pct_change(10)
        data['momentum_20'] = data['Close'].pct_change(20)
        
        # Volatility
        data['volatility_10'] = data['Close'].pct_change().rolling(10).std()
        data['volatility_20'] = data['Close'].pct_change().rolling(20).std()
        
        # Support and Resistance
        data['resistance'] = data['High'].rolling(20).max()
        data['support'] = data['Low'].rolling(20).min()
        data['price_vs_resistance'] = data['Close'] / data['resistance']
        data['price_vs_support'] = data['Close'] / data['support']
        
        return data
    
    def calculate_trend_strength(self, data):
        """Calcule la force de la tendance"""
        if len(data) < 20:
            return 0
        
        # Tendance basée sur les moyennes mobiles
        sma_trend = 0
        if data['sma_5'].iloc[-1] > data['sma_10'].iloc[-1] > data['sma_20'].iloc[-1]:
            sma_trend = 1  # Tendance haussière
        elif data['sma_5'].iloc[-1] < data['sma_10'].iloc[-1] < data['sma_20'].iloc[-1]:
            sma_trend = -1  # Tendance baissière
        
        # Force de la tendance basée sur la pente
        sma_5_slope = (data['sma_5'].iloc[-1] - data['sma_5'].iloc[-5]) / data['sma_5'].iloc[-5]
        sma_20_slope = (data['sma_20'].iloc[-1] - data['sma_20'].iloc[-5]) / data['sma_20'].iloc[-5]
        
        trend_strength = (sma_5_slope + sma_20_slope) / 2
        
        return sma_trend * abs(trend_strength)
    
    def calculate_momentum_score(self, data):
        """Calcule le score de momentum"""
        if len(data) < 20:
            return 0
        
        momentum_score = 0
        
        # RSI momentum
        rsi_14 = data['rsi_14'].iloc[-1]
        rsi_21 = data['rsi_21'].iloc[-1]
        rsi_30 = data['rsi_30'].iloc[-1]
        
        if rsi_14 < 30 and rsi_21 < 35:
            momentum_score += 0.3  # Survente
        elif rsi_14 > 70 and rsi_21 > 65:
            momentum_score -= 0.3  # Surachat
        
        # MACD momentum
        macd = data['macd'].iloc[-1]
        macd_signal = data['macd_signal'].iloc[-1]
        macd_hist = data['macd_histogram'].iloc[-1]
        
        if macd > macd_signal and macd_hist > 0:
            momentum_score += 0.2
        elif macd < macd_signal and macd_hist < 0:
            momentum_score -= 0.2
        
        # Stochastic momentum
        stoch_k = data['stoch_k'].iloc[-1]
        stoch_d = data['stoch_d'].iloc[-1]
        
        if stoch_k < 20 and stoch_d < 20:
            momentum_score += 0.2
        elif stoch_k > 80 and stoch_d > 80:
            momentum_score -= 0.2
        
        # Price momentum
        momentum_5 = data['momentum_5'].iloc[-1]
        momentum_10 = data['momentum_10'].iloc[-1]
        
        if momentum_5 > 0.02 and momentum_10 > 0.01:
            momentum_score += 0.3
        elif momentum_5 < -0.02 and momentum_10 < -0.01:
            momentum_score -= 0.3
        
        return momentum_score
    
    def calculate_volume_score(self, data):
        """Calcule le score de volume"""
        if len(data) < 20:
            return 0
        
        volume_ratio = data['volume_ratio'].iloc[-1]
        volume_trend = data['volume_price_trend'].iloc[-1]
        
        volume_score = 0
        
        if volume_ratio > 1.5:
            volume_score += 0.3  # Volume élevé
        elif volume_ratio < 0.5:
            volume_score -= 0.2  # Volume faible
        
        if volume_trend > 0:
            volume_score += 0.2  # Volume avec prix haussier
        else:
            volume_score -= 0.1  # Volume avec prix baissier
        
        return volume_score
    
    def calculate_volatility_score(self, data):
        """Calcule le score de volatilité"""
        if len(data) < 20:
            return 0
        
        volatility_10 = data['volatility_10'].iloc[-1]
        volatility_20 = data['volatility_20'].iloc[-1]
        
        # Volatilité normale est préférable
        if 0.01 < volatility_10 < 0.03 and 0.01 < volatility_20 < 0.03:
            return 0.2  # Volatilité normale
        elif volatility_10 > 0.05 or volatility_20 > 0.05:
            return -0.3  # Volatilité trop élevée
        else:
            return 0
    
    def calculate_support_resistance_score(self, data):
        """Calcule le score de support/résistance"""
        if len(data) < 20:
            return 0
        
        current_price = data['Close'].iloc[-1]
        resistance = data['resistance'].iloc[-1]
        support = data['support'].iloc[-1]
        
        score = 0
        
        # Proximité de la résistance
        if current_price > resistance * 0.98:
            score -= 0.3  # Près de la résistance
        
        # Proximité du support
        if current_price < support * 1.02:
            score += 0.3  # Près du support
        
        # Position dans la range
        range_position = (current_price - support) / (resistance - support)
        if range_position > 0.8:
            score -= 0.2  # Près du haut de la range
        elif range_position < 0.2:
            score += 0.2  # Près du bas de la range
        
        return score
    
    def analyze_market_advanced(self):
        """Analyse avancée du marché"""
        current_price = self.get_current_price()
        if current_price is None:
            return {
                'action': 'ERREUR',
                'reasoning': 'Impossible de récupérer les données',
                'confidence': 0
            }
        
        # Récupérer les données historiques
        data = self.get_historical_data(period='1y', interval='1d')
        if data is None or len(data) < 50:
            return {
                'action': 'ATTENDRE',
                'reasoning': 'Données insuffisantes pour l\'analyse avancée',
                'confidence': 0.3
            }
        
        # Calculer les indicateurs avancés
        data = self.calculate_advanced_indicators(data)
        if data is None:
            return {
                'action': 'ATTENDRE',
                'reasoning': 'Erreur dans le calcul des indicateurs',
                'confidence': 0.3
            }
        
        # Calculer les scores
        trend_strength = self.calculate_trend_strength(data)
        momentum_score = self.calculate_momentum_score(data)
        volume_score = self.calculate_volume_score(data)
        volatility_score = self.calculate_volatility_score(data)
        support_resistance_score = self.calculate_support_resistance_score(data)
        
        # Score total
        total_score = (trend_strength * 0.3 + 
                      momentum_score * 0.25 + 
                      volume_score * 0.2 + 
                      volatility_score * 0.15 + 
                      support_resistance_score * 0.1)
        
        # Décision basée sur le score total
        if total_score > 0.6 and self.current_position <= 0:
            self.current_position = 1
            self.entry_price = current_price
            return {
                'action': 'ACHETER',
                'reasoning': f'Score total: {total_score:.3f} (Tendance: {trend_strength:.3f}, Momentum: {momentum_score:.3f}, Volume: {volume_score:.3f})',
                'confidence': min(0.95, 0.6 + total_score * 0.3),
                'price': current_price,
                'scores': {
                    'total': total_score,
                    'trend': trend_strength,
                    'momentum': momentum_score,
                    'volume': volume_score,
                    'volatility': volatility_score,
                    'support_resistance': support_resistance_score
                }
            }
        elif total_score < -0.6 and self.current_position >= 0:
            self.current_position = -1
            self.entry_price = current_price
            return {
                'action': 'VENDRE',
                'reasoning': f'Score total: {total_score:.3f} (Tendance: {trend_strength:.3f}, Momentum: {momentum_score:.3f}, Volume: {volume_score:.3f})',
                'confidence': min(0.95, 0.6 + abs(total_score) * 0.3),
                'price': current_price,
                'scores': {
                    'total': total_score,
                    'trend': trend_strength,
                    'momentum': momentum_score,
                    'volume': volume_score,
                    'volatility': volatility_score,
                    'support_resistance': support_resistance_score
                }
            }
        else:
            return {
                'action': 'ATTENDRE',
                'reasoning': f'Score total: {total_score:.3f} (Tendance: {trend_strength:.3f}, Momentum: {momentum_score:.3f}, Volume: {volume_score:.3f})',
                'confidence': 0.4 + abs(total_score) * 0.2,
                'price': current_price,
                'scores': {
                    'total': total_score,
                    'trend': trend_strength,
                    'momentum': momentum_score,
                    'volume': volume_score,
                    'volatility': volatility_score,
                    'support_resistance': support_resistance_score
                }
            }
    
    def get_status(self):
        """Retourne le statut actuel"""
        current_price = self.get_current_price()
        if current_price is None:
            return "❌ Impossible de récupérer le prix"
        
        status = f"💰 Prix {self.symbol}: ${current_price:.2f}\n"
        status += f"📊 Position: {self.current_position}\n"
        
        if self.current_position != 0:
            pnl = ((current_price - self.entry_price) / self.entry_price * 100)
            status += f"💵 P&L: {pnl:+.2f}%\n"
        
        return status

def demo_advanced_agent():
    """Démonstration de l'agent avancé"""
    print("🚀 DÉMONSTRATION - AGENT DE TRADING AVANCÉ")
    print("=" * 70)
    
    # Créer l'agent
    print("📱 Initialisation de l'agent avancé pour AAPL...")
    agent = AdvancedTradingAgent('AAPL')
    
    # Prendre quelques décisions
    print("\n🎯 PRISE DE DÉCISIONS AVANCÉES")
    print("-" * 50)
    
    for i in range(3):
        print(f"\n🔄 Décision #{i+1}")
        decision = agent.analyze_market_advanced()
        
        print(f"🎯 Action: {decision['action']}")
        print(f"📝 Raison: {decision['reasoning']}")
        print(f"🎲 Confiance: {decision['confidence']:.1%}")
        
        if 'scores' in decision:
            scores = decision['scores']
            print(f"📊 Scores: Total={scores['total']:.3f}, Tendance={scores['trend']:.3f}, Momentum={scores['momentum']:.3f}")
        
        if 'price' in decision:
            print(f"💰 Prix: ${decision['price']:.2f}")
        
        # Attendre 2 secondes entre les décisions
        if i < 2:
            print("⏳ Attente de 2 secondes...")
            time.sleep(2)
    
    # Afficher le statut final
    print("\n📊 STATUT FINAL")
    print("-" * 30)
    status = agent.get_status()
    print(status)
    
    print("\n✅ DÉMONSTRATION TERMINÉE!")
    print("L'agent avancé est prêt avec une corrélation améliorée!")

def run_prediction_analysis(symbol='AAPL'):
    """
    Lance une analyse de prédiction automatique pour déterminer s'il faut attendre ou agir
    """
    print("🔮 ANALYSE DE PRÉDICTION AUTOMATIQUE - DÉCISION D'INVESTISSEMENT")
    print("=" * 80)
    print(f"📊 Symbole analysé: {symbol}")
    print(f"🕐 Heure d'analyse: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print("=" * 80)
    
    # Créer l'agent avancé
    print("🤖 Initialisation de l'agent de trading avancé...")
    agent = AdvancedTradingAgent(symbol)
    
    # Effectuer l'analyse
    print("\n🔍 ANALYSE EN COURS...")
    print("-" * 50)
    
    decision = agent.analyze_market_advanced()
    
    # Afficher les résultats
    print("\n📊 RÉSULTATS DE L'ANALYSE")
    print("=" * 50)
    
    # Prix actuel
    current_price = agent.get_current_price()
    print(f"💰 Prix actuel {symbol}: ${current_price:.2f}")
    
    # Décision principale
    action = decision['action']
    confidence = decision['confidence']
    reasoning = decision['reasoning']
    
    print(f"\n🎯 RECOMMANDATION: {action}")
    print(f"📝 Justification: {reasoning}")
    print(f"🎲 Niveau de confiance: {confidence:.1%}")
    
    # Scores détaillés si disponibles
    if 'scores' in decision:
        scores = decision['scores']
        print(f"\n📈 ANALYSE DÉTAILLÉE:")
        print(f"   • Score total: {scores['total']:.3f}")
        print(f"   • Tendance: {scores['trend']:.3f}")
        print(f"   • Momentum: {scores['momentum']:.3f}")
        print(f"   • Volume: {scores['volume']:.3f}")
        print(f"   • Volatilité: {scores['volatility']:.3f}")
        print(f"   • Support/Résistance: {scores['support_resistance']:.3f}")
    
    # Interprétation de la recommandation
    print(f"\n💡 INTERPRÉTATION:")
    if action == 'ACHETER':
        print("✅ SIGNAL D'ACHAT DÉTECTÉ")
        print("   → Les indicateurs techniques suggèrent une opportunité d'achat")
        print("   → Conditions favorables pour entrer en position longue")
        print("   → Recommandation: AGIR - Procéder à l'achat")
    elif action == 'VENDRE':
        print("❌ SIGNAL DE VENTE DÉTECTÉ")
        print("   → Les indicateurs techniques suggèrent une opportunité de vente")
        print("   → Conditions favorables pour entrer en position courte")
        print("   → Recommandation: AGIR - Procéder à la vente")
    else:  # ATTENDRE
        print("⏳ SIGNAL D'ATTENTE DÉTECTÉ")
        print("   → Les conditions du marché ne sont pas optimales")
        print("   → Mélange de signaux ou conditions incertaines")
        print("   → Recommandation: ATTENDRE - Surveiller l'évolution")
    
    # Niveau de confiance
    print(f"\n🎲 ÉVALUATION DE LA CONFIANCE:")
    if confidence >= 0.8:
        print("🟢 CONFIANCE ÉLEVÉE - Recommandation forte")
    elif confidence >= 0.6:
        print("🟡 CONFIANCE MODÉRÉE - Recommandation modérée")
    else:
        print("🔴 CONFIANCE FAIBLE - Prudence recommandée")
    
    # Statut de position
    print(f"\n📊 STATUT DE POSITION:")
    status = agent.get_status()
    print(status)
    
    # Recommandation finale
    print(f"\n🏁 RECOMMANDATION FINALE:")
    print("=" * 50)
    
    if action == 'ACHETER' and confidence >= 0.6:
        print("✅ AGIR - ACHETER")
        print("   Les conditions sont favorables pour un achat")
        print("   Niveau de confiance suffisant pour agir")
    elif action == 'VENDRE' and confidence >= 0.6:
        print("✅ AGIR - VENDRE")
        print("   Les conditions sont favorables pour une vente")
        print("   Niveau de confiance suffisant pour agir")
    else:
        print("⏳ ATTENDRE")
        print("   Les conditions ne sont pas optimales pour agir")
        print("   Surveiller l'évolution du marché")
    
    print(f"\n🕐 Analyse terminée à: {datetime.now().strftime('%H:%M:%S')}")
    print("=" * 80)
    
    return decision

def main():
    """Fonction principale - Menu complet"""
    print("🤖 SCRIPT COMPLET AUTONOME - AGENT DE TRADING AVANCÉ")
    print("=" * 70)
    
    while True:
        print("\n📋 MENU PRINCIPAL")
        print("1. 🎯 Démonstration rapide (3 décisions)")
        print("2. 🔮 Prédiction complète pour décision d'investissement")
        print("3. 🚪 Quitter")
        
        choice = input("\nChoisissez une option (1-3): ")
        
        if choice == '1':
            demo_advanced_agent()
            
        elif choice == '2':
            symbol = input("Entrez le symbole boursier (ex: AAPL, MSFT, GOOGL) ou appuyez sur Entrée pour AAPL: ").upper()
            if not symbol:
                symbol = 'AAPL'
            run_prediction_analysis(symbol)
            
        elif choice == '3':
            print("👋 Au revoir!")
            break
            
        else:
            print("❌ Option invalide")

if __name__ == "__main__":
    main()
"""
            Démonstration de l'Agent de Trading Avancé
        """
import warnings
warnings.filterwarnings('ignore')

from advanced_trading_agent import AdvancedTradingAgent
import time

def demo_advanced_agent():
    """Démonstration de l'agent avancé"""
    print("🚀 DÉMONSTRATION - AGENT DE TRADING AVANCÉ")
    print("=" * 70)
    
    # Créer l'agent
    print("📱 Initialisation de l'agent avancé pour AAPL...")
    agent = AdvancedTradingAgent('AAPL')
    
    # Prendre quelques décisions
    print("\n🎯 PRISE DE DÉCISIONS AVANCÉES")
    print("-" * 50)
    
    for i in range(3):
        print(f"\n🔄 Décision #{i+1}")
        decision = agent.analyze_market_advanced()
        
        print(f"🎯 Action: {decision['action']}")
        print(f"📝 Raison: {decision['reasoning']}")
        print(f"🎲 Confiance: {decision['confidence']:.1%}")
        
        if 'scores' in decision:
            scores = decision['scores']
            print(f"📊 Scores: Total={scores['total']:.3f}, Tendance={scores['trend']:.3f}, Momentum={scores['momentum']:.3f}")
        
        if 'price' in decision:
            print(f"💰 Prix: ${decision['price']:.2f}")
        
        # Attendre 2 secondes entre les décisions
        if i < 2:
            print("⏳ Attente de 2 secondes...")
            time.sleep(2)
    
    # Afficher le statut final
    print("\n📊 STATUT FINAL")
    print("-" * 30)
    status = agent.get_status()
    print(status)
    
    print("\n✅ DÉMONSTRATION TERMINÉE!")
    print("L'agent avancé est prêt avec une corrélation améliorée!")
    
    # Proposer de lancer le monitoring
    choice = input("\nVoulez-vous lancer le monitoring avancé en temps réel ? (o/n): ")
    if choice.lower() in ['o', 'oui', 'y', 'yes']:
        interval = input("Intervalle en secondes (défaut: 30): ")
        try:
            interval = int(interval) if interval else 30
        except:
            interval = 30
        agent.run_monitoring(interval)

if __name__ == "__main__":
    demo_advanced_agent()

🤖 SCRIPT COMPLET AUTONOME - AGENT DE TRADING AVANCÉ

📋 MENU PRINCIPAL
1. 🎯 Démonstration rapide (3 décisions)
2. 🔮 Prédiction complète pour décision d'investissement
3. 🚪 Quitter
🚀 DÉMONSTRATION - AGENT DE TRADING AVANCÉ
📱 Initialisation de l'agent avancé pour AAPL...

🎯 PRISE DE DÉCISIONS AVANCÉES
--------------------------------------------------

🔄 Décision #1
🎯 Action: ATTENDRE
📝 Raison: Score total: 0.155 (Tendance: 0.000, Momentum: 0.500, Volume: 0.000)
🎲 Confiance: 43.1%
📊 Scores: Total=0.155, Tendance=0.000, Momentum=0.500
💰 Prix: $259.83
⏳ Attente de 2 secondes...

🔄 Décision #2
🎯 Action: ATTENDRE
📝 Raison: Score total: 0.155 (Tendance: 0.000, Momentum: 0.500, Volume: 0.000)
🎲 Confiance: 43.1%
📊 Scores: Total=0.155, Tendance=0.000, Momentum=0.500
💰 Prix: $259.83
⏳ Attente de 2 secondes...

🔄 Décision #3
🎯 Action: ATTENDRE
📝 Raison: Score total: 0.155 (Tendance: 0.000, Momentum: 0.500, Volume: 0.000)
🎲 Confiance: 43.1%
📊 Scores: Total=0.155, Tendance=0.000, Momentum=0.500
💰 Pri

ModuleNotFoundError: No module named 'advanced_trading_agent'